<a href="https://colab.research.google.com/github/tarushsingh09/STYLE_GAN/blob/main/E21CSEU0974__LAB_9__IMD_STYLE_GAN_TARUSH_SINGH_EB_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#E21CSEU0974
#TARUSH SINGH
#EB-13
#G-2
#INTELLIGENT MODEL DESIGN USING AI LAB 9
##STYLE GAN



## Task-to-Do
*   Understand the components of StyleGAN that differ from the traditional GAN.
*   Implement the components of StyleGAN(1.5 Marks)
*   Compare PSO method with keras tuner.

#Task 1: Understand the Components of StyleGAN:

##StyleGAN introduces several components that distinguish it from traditional GANs. Some key components include:

*   Mapping Network: StyleGAN uses a mapping network to transform a latent vector into an intermediate latent space, which is then used to generate images.
*   Style Mixing: Allows the model to combine different styles at different resolutions during the generation process.
*   Adaptive Instance Normalization (AdaIN): Modifies the mean and standard deviation of the intermediate features to incorporate style information.
*   Noise Injection: Adds controlled noise to the input of each layer, enhancing variety in generated images.







#Task 2: Implement Components of StyleGAN


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2D, UpSampling2D, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Mapping Network
def mapping_network(latent_dim, hidden_dim, num_layers):
    inputs = Input(shape=(latent_dim,))
    x = inputs
    for _ in range(num_layers):
        x = Dense(hidden_dim, activation='relu')(x)
    outputs = Dense(latent_dim)(x)
    return Model(inputs, outputs, name='mapping_network')

# Generator
def stylegan_generator(latent_dim, hidden_dim, num_layers, resolution):
    mapping = mapping_network(latent_dim, hidden_dim, num_layers)

    noise_inputs = Input(shape=(resolution, resolution, 1))
    latent_inputs = Input(shape=(latent_dim,))

    x = mapping(latent_inputs)

    # AdaIN and Noise Injection
    x = Lambda(lambda x: x[0] * K.expand_dims(K.expand_dims(K.expand_dims(x[1], axis=1), axis=1), axis=1) + x[2])([x, noise_inputs, x])

    # Upsampling layers...

    outputs = Conv2D(3, kernel_size=(1, 1), activation='tanh')(x)

    return Model([noise_inputs, latent_inputs], outputs, name='stylegan_generator')

# Instantiate the generator
latent_dim = 128
hidden_dim = 256
num_layers = 8
resolution = 256

generator = stylegan_generator(latent_dim, hidden_dim, num_layers, resolution)

# Print the model summary
generator.summary()

Model: "stylegan_generator"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 mapping_network (Functiona  (None, 128)                  526464    ['input_3[0][0]']             
 l)                                                                                               
                                                                                                  
 input_2 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 lambda (Lambda)             (None, 1, 1, 1, 256, 256,    0         ['mapping_net

#Task 3: Compare PSO with Keras Tuner


In [11]:
pip install pyswarm

In [12]:
import time
import numpy as np
from sklearn.datasets import make_classification  # Import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a basic MLP model
def build_model(hidden_layer_sizes, learning_rate):
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=learning_rate, random_state=42)
    return model

# Train and evaluate a model with default hyperparameters
model = build_model(hidden_layer_sizes=(100, 100), learning_rate=0.001)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_default = accuracy_score(y_test, y_pred)
print("Default Model Accuracy:", accuracy_default)

Default Model Accuracy: 0.81


In [22]:
import time
import pyswarm
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.extmath import safe_sparse_dot

# Define a fitness function for PSO
def fitness_function_pso(x):
    hidden_layer_sizes = (int(x[0]), int(x[1]))
    learning_rate = x[2]

    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=learning_rate, random_state=42)

    n_epochs = 10  # You can adjust the number of epochs

    best_loss = float("inf")
    n_no_improvement = 0
    patience = 5  # Adjust the patience as needed

    for epoch in range(n_epochs):
        model.partial_fit(X_train, y_train, classes=np.unique(y_train))
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        val_loss = -accuracy  # Inverse accuracy as PSO minimizes
        if val_loss < best_loss:
            best_loss = val_loss
            n_no_improvement = 0
        else:
            n_no_improvement += 1
            if n_no_improvement >= patience:
                break  # Early stopping

    return best_loss

# PSO hyperparameter search space (bounds for hidden layers and learning rate)
lb = [50, 50, 0.0001]
ub = [200, 200, 0.01]

# Measure the time taken by PSO
start_time_pso = time.time()

# Use PSO to optimize hyperparameters
x_opt_pso, f_opt_pso = pyswarm.pso(fitness_function_pso, lb, ub, swarmsize=10, maxiter=20)
# Calculate the time taken by PSO
pso_time = time.time() - start_time_pso
# Calculate predictions for the PSO model
y_pred_pso = model.predict(X_test)
accuracy_pso = accuracy_score(y_test, y_pred_pso)

Stopping search: maximum iterations reached --> 20


In [23]:
print("Optimized Hyperparameters (PSO):")
print("Hidden Layer Sizes:", (int(x_opt_pso[0]), int(x_opt_pso[1])))
print("Learning Rate:", x_opt_pso[2])

Optimized Hyperparameters (PSO):
Hidden Layer Sizes: (148, 105)
Learning Rate: 0.0021932045272550823


In [27]:
import time
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function to build the Keras model with hyperparameters
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=50, max_value=200, step=10), activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(units=hp.Int('units_2', min_value=50, max_value=200, step=10), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-4, 1e-3, 1e-2])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model
# Measure the time taken by Keras Tuner
start_time_keras_tuner = time.time()

# Instantiate the Keras Tuner RandomSearch tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=10, directory='keras_tuner_dir', project_name='my_tuner')

# Search for the best hyperparameters using Keras Tuner
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model using the best hyperparameters
best_model_keras_tuner = tuner.hypermodel.build(best_hps)

# Train the best model
best_model_keras_tuner.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Calculate the time taken by Keras Tuner
keras_tuner_time = time.time() - start_time_keras_tuner

# Predict probabilities using the model
y_prob_keras_tuner = best_model_keras_tuner.predict(X_test)

# Convert probabilities to binary class predictions (e.g., threshold at 0.5)
y_pred_keras_tuner = (y_prob_keras_tuner > 0.5).astype(int)

# Calculate the accuracy based on binary predictions
accuracy_keras_tuner = accuracy_score(y_test, y_pred_keras_tuner)

Reloading Tuner from keras_tuner_dir/my_tuner/tuner0.json
Epoch 1/10
25/25 [==============================] - 2s 27ms/step - loss: 0.4278 - accuracy: 0.8275 - val_loss: 0.3536 - val_accuracy: 0.8400
Epoch 2/10
25/25 [==============================] - 0s 9ms/step - loss: 0.2915 - accuracy: 0.8875 - val_loss: 0.3903 - val_accuracy: 0.8850
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2497 - accuracy: 0.9087 - val_loss: 0.3806 - val_accuracy: 0.8700
Epoch 4/10
25/25 [==============================] - 0s 8ms/step - loss: 0.2131 - accuracy: 0.9162 - val_loss: 0.4181 - val_accuracy: 0.8750
Epoch 5/10
25/25 [==============================] - 0s 12ms/step - loss: 0.1832 - accuracy: 0.9312 - val_loss: 0.4431 - val_accuracy: 0.8500
Epoch 6/10
25/25 [==============================] - 0s 9ms/step - loss: 0.1594 - accuracy: 0.9413 - val_loss: 0.4762 - val_accuracy: 0.8100
Epoch 7/10
25/25 [==============================] - 0s 11ms/step - loss: 0.1544 - accuracy: 0.9325 

In [28]:
# Print the optimized hyperparameters
print("Optimized Hyperparameters (Keras Tuner):")
print("Hidden Layer Sizes:", best_hps.get('units_1'), best_hps.get('units_2'))
print("Learning Rate:", best_hps.get('learning_rate'))

Optimized Hyperparameters (Keras Tuner):
Hidden Layer Sizes: 70 200
Learning Rate: 0.01


In [29]:
# Print and compare results
print("PSO Time:", pso_time)
print("Keras Tuner Time:", keras_tuner_time)
print("PSO Accuracy:", accuracy_pso)
print("Keras Tuner Accuracy:", accuracy_keras_tuner)

PSO Time: 31.654032707214355
Keras Tuner Time: 4.832805633544922
PSO Accuracy: 0.81
Keras Tuner Accuracy: 0.825
